In [1]:
from nhlpy.api.query.builder import QueryBuilder, QueryContext, QueryBase
from nhlpy.nhl_client import NHLClient
from nhlpy.api.query.filters.draft import DraftQuery
from nhlpy.api.query.filters.season import SeasonQuery
from nhlpy.api.query.filters.game_type import GameTypeQuery
from nhlpy.api.query.filters.position import PositionQuery, PositionTypes
import polars as pl

In [2]:
client = NHLClient(verbose=True)

filters: list[QueryBase] = [
    GameTypeQuery(game_type="2"),
    # DraftQuery(year="2020", draft_round="2"),
    # SeasonQuery(season_start="20012002", season_end="20242025"),
    PositionQuery(position=PositionTypes.ALL_FORWARDS),
]

query_builder = QueryBuilder()
query_context: QueryContext = query_builder.build(filters=filters)

data = client.stats.skater_stats_with_query_context(
    report_type='summary',
    query_context=query_context,
    aggregate=True
)

data = pl.DataFrame(data['data'])

crosby = data.filter(pl.col('skaterFullName') == 'Sidney Crosby')

ppg = crosby.select('pointsPerGame').item()
print(ppg)

INFO:httpx:HTTP Request: GET https://api.nhle.com/stats/rest/en/skater/summary?isAggregate=true&isGame=false&start=0&limit=25&factCayenneExp=gamesPlayed%3E%3D1&sort=%5B%7B%22property%22%3A+%22points%22%2C+%22direction%22%3A+%22DESC%22%7D%2C+%7B%22property%22%3A+%22gamesPlayed%22%2C+%22direction%22%3A+%22ASC%22%7D%2C+%7B%22property%22%3A+%22playerId%22%2C+%22direction%22%3A+%22ASC%22%7D%5D&cayenneExp=gameTypeId%3D2+and+%28positionCode%3D%27L%27+or+positionCode%3D%27R%27+or+positionCode%3D%27C%27%29 "HTTP/1.1 200 OK"


1.24778
